## Setup and Import Libraries

In [10]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [2]:
load_dotenv()

True

In [3]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [5]:
llm = ChatGroq(model="Gemma2-9b-It")

## Messages

In [8]:
llm.invoke(
    [
        HumanMessage(content="Hi, My name is Saad Tariq and I am a Data Scientist")
    ]
)

AIMessage(content="Hello Saad Tariq! It's nice to meet you.\n\nBeing a Data Scientist is a fascinating field. What kind of work are you involved in? Do you have any interesting projects you'd like to talk about? \n\nI'm always eager to learn more about the work people are doing with data.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 23, 'total_tokens': 91, 'completion_time': 0.123636364, 'prompt_time': 0.002107285, 'queue_time': 0.018710784, 'total_time': 0.125743649}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-84ff85e1-7a87-48ff-ac81-1b05b118e0be-0', usage_metadata={'input_tokens': 23, 'output_tokens': 68, 'total_tokens': 91})

In [9]:
llm.invoke(
    [
        HumanMessage(content="Hi, My name is Saad Tariq and I am a Data Scientist"),
        AIMessage(content="Hello Saad Tariq! It's nice to meet you.\n\nBeing a Data Scientist is a fascinating field. What kind of work are you involved in?"),
        HumanMessage(content="What is my name and What I do?")
    ]
)

AIMessage(content="You said your name is Saad Tariq and that you are a Data Scientist. \n\nIs there anything else you'd like to tell me about yourself or your work? I'm eager to learn more! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 74, 'total_tokens': 123, 'completion_time': 0.089090909, 'prompt_time': 0.003853789, 'queue_time': 0.018962999, 'total_time': 0.092944698}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-24275743-fdf5-4bdf-9105-e740099d0efe-0', usage_metadata={'input_tokens': 74, 'output_tokens': 49, 'total_tokens': 123})

## Message History

We can use a Message History class to wrap our model and make it stateful. This will keep track of the inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input.

In [11]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
        
    return store[session_id]

In [12]:
config = {
    "configurable": {"session_id": "chat1"}
}

In [13]:
with_message_history = RunnableWithMessageHistory(llm, get_session_history)

In [15]:
response = with_message_history.invoke(
    [
        HumanMessage(content="Hi, My name is Saad Tariq and I am a Data Scientist"),
    ],
    config=config
)

In [16]:
response.content

"Hello Saad Tariq!  \n\nIt's great to meet you. Being a Data Scientist is a really interesting profession. \n\nWhat kind of projects are you working on these days?  Do you have any specific questions for me or are you just saying hello? 😊  I'm happy to chat about data science, help with code, or just have a friendly conversation.  \n\n"

In [17]:
response = with_message_history.invoke(
    [
        HumanMessage(content="What is my name"),
    ],
    config=config
)

response.content

'Your name is Saad Tariq, as you told me earlier.  😊  \n\nIs there anything else I can help you with?\n'

## Changing the Config

In [ ]:
config = {
    "configurable": {"session_id": "chat1"}
}